In [222]:
from collections import namedtuple

import requests_cache

RoundStats = namedtuple('RoundStats', ['blackbox', 'playercounts', 'metadata'])

session = requests_cache.CachedSession('api_paradisestation_org')

def get_roundstats(round_id):
    blackbox = session.get(f"https://api.paradisestation.org/stats/blackbox/{round_id}")
    playercounts = session.get(f"https://api.paradisestation.org/stats/playercounts/{round_id}")
    metadata = session.get(f"https://api.paradisestation.org/stats/metadata/{round_id}")
    return RoundStats(blackbox=blackbox.json(), playercounts=playercounts.json(), metadata=metadata.json())

In [151]:
import json
import toml

In [145]:
rd = get_roundstats(36833)
jp = [x for x in rd.blackbox if x.get('key_name') == 'job_preferences'][0]
jprd = json.loads(jp['raw_data'])
manifest = [x for x in rd.blackbox if x.get('key_name') == 'manifest'][0]
mrd = json.loads(manifest['raw_data'])

In [24]:
class RoundStatsMunger:
    def __init__(self, roundstats):
        self.roundstats = roundstats
        self._cache = dict()

    def _get_stat(self, keyname):
        if keyname not in self._cache:
            for x in self.roundstats.blackbox:
                if x.get('key_name') == keyname:
                    self._cache[keyname] = json.loads(x['raw_data'])['data']
            
        return self._cache[keyname]

    def raw_manifest(self):
        pass

In [25]:
rsm = RoundStatsMunger(rd)

In [33]:
rd.metadata

{'round_id': 36833,
 'init_datetime': '2023-08-07T07:54:10',
 'start_datetime': '2023-08-07T07:58:54',
 'shutdown_datetime': '2023-08-07T09:18:17',
 'end_datetime': '2023-08-07T09:16:53',
 'commit_hash': '1f7ec479523a098587c5a969d901fc9e6fac26d5',
 'game_mode': 'cult',
 'game_mode_result': 'cult win - cult win',
 'end_state': 'proper completion',
 'map_name': 'Cyberiad',
 'server_id': 'paradise_main'}

In [34]:
max_count = 100
total_count = 0

In [37]:
rl = session.get("https://api.paradisestation.org/stats/roundlist").json()

In [83]:
offset = min(roundstats.keys()) - 50
rl = session.get(f"https://api.paradisestation.org/stats/roundlist?offset={offset}").json()

In [85]:
rounds = sorted(set([x['round_id'] for x in rl]))

In [77]:
min(roundstats.keys())

36833

In [42]:
roundstats = dict()

In [43]:
import random

In [86]:
for r_id in rounds:
    if r_id in roundstats:
        continue
    roundstats[r_id] = get_roundstats(r_id)

In [46]:
random.choice(rounds)

36878

In [50]:
blackbox_keys = set()

In [88]:
for rd in roundstats.values():
    for x in rd.blackbox:
        blackbox_keys.add((x.get('key_name'), x.get('key_type'), x.get('version')))

In [89]:
len(blackbox_keys)

76

In [55]:
rs = roundstats[36878]

In [66]:
rsm = RoundStatsMunger(rs)

In [154]:
constrained_rounds

{(36734, 72),
 (36736, 71),
 (36737, 74),
 (36752, 69),
 (36774, 76),
 (36775, 70),
 (36776, 71),
 (36777, 72),
 (36876, 69),
 (36887, 71)}

In [136]:
for x in constrained_rounds:
    r_id = x[0]
    rs = roundstats[r_id]
    print(rs.metadata['game_mode'])

Trifecta
traitor+changeling


In [146]:
slot_amount_names = set([x['name'] for x in t['job_configuration']['job_slot_amounts']])

In [147]:
highcount_names = {x for x in highcounts.keys()}

In [148]:
highcount_names - slot_amount_names

{'Nanotrasen Navy Officer',
 'Solar Federation General',
 'Special Operations Officer',
 'Syndicate Officer'}

What we are trying to prove: That by opening up highpop job slots before job assignment, more people who ready up roundstart will get the roles they desire, as opposed to not readying up and joining immediately after highpop slots are enabled after roundstart.
 
The number of people who actually readied up for a given round (and weren't just idling/observing) is the number of available job preference tallies. We use the Assistant job as a sentinel since it is either set to never, or low, which means the user wishes to spawn in as an assistant.

Also remember that the highpop trigger is _the total number of logged in clients_, irrespective of status, role, or state. 

The highpop and lowpop job slots are given by the config.toml, after it's fully fleshed out to include all available jobs (which it isn't currently).

In [152]:
t = toml.load(open('D:/ExternalRepos/third_party/Paradise/config/config.toml'))
lp = sum([x['lowpop'] for x in t['job_configuration']['job_slot_amounts'] if x['lowpop'] > 0])
hp = sum([x['highpop'] for x in t['job_configuration']['job_slot_amounts'] if x['highpop'] > 0])

(lp, hp)

(68, 75)

So highpop adds 7 more slots to various roles.

Now we need to find instances of rounds where there were more readied players than slots, _and what jobs were desired versus assigned_. We also want to ignore rounds that could randomly assign a lot of people to unassignable roles (such a nukies, which can take 5 people, which would obviously throw this off). We also only care about rounds where the roundstart population would have trigged highpop slots.

In [201]:
from collections import namedtuple

RoundJobStats = namedtuple('RoundJobStats', ['round_id', 'game_mode', 'ready_count', 'roundstart_client_count', 'roundstart_nonasst_job_count']) 

constrained_rounds = list()

for r_id, rs in sorted(roundstats.items()):
    rsm = RoundStatsMunger(rs)
    rjs = RoundJobStats(
        round_id = r_id,
        game_mode = rs.metadata['game_mode'],
        ready_count = rsm._get_stat('job_preferences')['Assistant']['never'],
        roundstart_client_count = sorted(rs.playercounts.items())[0][-1],
        roundstart_nonasst_job_count = sum([y.get("roundstart", 0) for x,y in rsm._get_stat('manifest').items() if x != 'Assistant']),
    )
 #   print(rjs)
    if rjs.ready_count >= lp and rjs.roundstart_client_count >= 80:
        constrained_rounds.append(rjs)
#    print(f"{rjs.round_id}: {rjs.ready_count - rjs.roundstart_nonasst_job_count}")

constrained_rounds

[RoundJobStats(round_id=36737, game_mode='Trifecta', ready_count=68, roundstart_client_count=113, roundstart_nonasst_job_count=64),
 RoundJobStats(round_id=36774, game_mode='traitor+changeling', ready_count=71, roundstart_client_count=98, roundstart_nonasst_job_count=64)]

So let's collect everything we know about a round. We want:

- The roundstart client population.
- The number of readied players on roundstart.
- The "highcounts" for each role (number of readied players who set that job to high)
- The actual roundstart job assignment counts
- How many people actually spawned in, roundstart, that weren't assistants

Something we can't have data on is how many people know about this quirk in job assignment and intentionally do not ready up so that they can immediately latejoin with their desired role. That's okay for the time being, because what we want to use as our sample population is people who do set up their job preferences and ready up with the expectation they will be assigned a desirable role if available. That's the group we care about comparing against TMed rounds where highpop slots are enabled before assignment (while we do have latejoin stats they are aggregated across the entire round's runtime).

In [185]:
rs = roundstats[36737]
rsm = RoundStatsMunger(rs)

roundstart_client_population = sorted(rs.playercounts.items())[0][-1]
readied_player_count = rsm._get_stat('job_preferences')['Nanotrasen Navy Officer']['never']
prefcounts = dict()

for name, data in rsm._get_stat('job_preferences').items():
    ct = sum([y for x,y in data.items() if x in ('high', 'medium', 'low',)])
    if ct:
        prefcounts[name] = ct

roundstart_jobs = {x: y.get("roundstart", 0) for x,y in rsm._get_stat('manifest').items() if x != 'Assistant'}
roundstart_spawn_count = sum([y.get("roundstart", 0) for x,y in rsm._get_stat('manifest').items() if x != 'Assistant'])

In [176]:
roundstart_client_population, readied_player_count, roundstart_spawn_count

(113, 74, 64)

Okay so there were 113 clients on this round, 74 readied players, and 64 roundstart spawns. This is four less than the gross lowpop slot count, so four players got booted back to the lobby. Would those players have gotten in if the highpop map were enabled? Possibly. There could have been five high preferences for Captain but obviously that wouldn't change.

In [179]:
prefcounts

{'Assistant': 6,
 'Chief Engineer': 6,
 'Station Engineer': 30,
 'Life Support Specialist': 21,
 'Chief Medical Officer': 9,
 'Medical Doctor': 33,
 'Coroner': 19,
 'Chemist': 19,
 'Geneticist': 16,
 'Virologist': 18,
 'Psychiatrist': 13,
 'Paramedic': 24,
 'Research Director': 3,
 'Scientist': 32,
 'Roboticist': 22,
 'Head of Security': 8,
 'Warden': 9,
 'Detective': 20,
 'Security Officer': 34,
 'AI': 15,
 'Cyborg': 21,
 'Captain': 6,
 'Head of Personnel': 7,
 'Nanotrasen Representative': 5,
 'Blueshield': 11,
 'Magistrate': 2,
 'Internal Affairs Agent': 11,
 'Quartermaster': 19,
 'Cargo Technician': 27,
 'Shaft Miner': 35,
 'Bartender': 24,
 'Chef': 23,
 'Botanist': 21,
 'Clown': 19,
 'Mime': 16,
 'Janitor': 36,
 'Librarian': 14,
 'Barber': 9,
 'Explorer': 15,
 'Chaplain': 17}

Let's say someone wanted to play engineer. There are 5 lowpop and 6 highpop slots. 

In [186]:
roundstart_jobs

{'Medical Doctor': 3,
 'Explorer': 4,
 'Psychiatrist': 1,
 'Security Officer': 7,
 'Scientist': 5,
 'Chef': 1,
 'Clown': 1,
 'Cargo Technician': 2,
 'Mime': 1,
 'Virologist': 1,
 'Geneticist': 2,
 'Barber': 1,
 'Shaft Miner': 8,
 'Chief Medical Officer': 1,
 'Blueshield': 1,
 'Station Engineer': 2,
 'Research Director': 1,
 'Bartender': 1,
 'Cyborg': 2,
 'Janitor': 1,
 'Nanotrasen Representative': 1,
 'Head of Personnel': 1,
 'Chemist': 2,
 'Captain': 1,
 'Coroner': 1,
 'Internal Affairs Agent': 1,
 'Botanist': 2,
 'Paramedic': 1,
 'Life Support Specialist': 2,
 'Chaplain': 1,
 'Roboticist': 2,
 'Quartermaster': 1,
 'Detective': 1,
 'Librarian': 1,
 'Magistrate': 0,
 'Warden': 0,
 'AI': 0}

In [214]:
highpop_additions = dict()
for role in t['job_configuration']['job_slot_amounts']:
    hp = role.get('highpop', -1)
    lp = role.get('lowpop', -1)
    if hp > lp and lp > 0:
        highpop_additions[role.get('name')] = (lp, hp, hp - role.get('lowpop', 0))
        

In [215]:
highpop_additions

{'Life Support Specialist': (3, 4, 1),
 'Station Engineer': (5, 6, 1),
 'Medical Doctor': (5, 6, 1),
 'Scientist': (6, 7, 1),
 'Security Officer': (8, 9, 1),
 'Janitor': (1, 2, 1),
 'Cargo Technician': (3, 4, 1)}

In [216]:
prefcounts2 = dict()

for name in highpop_additions.keys():
    ct = sum([y for x,y in rsm._get_stat('job_preferences')[name].items() if x == 'high'])
    if ct:
        prefcounts2[name] = ct

prefcounts2

{'Station Engineer': 1,
 'Medical Doctor': 2,
 'Scientist': 3,
 'Security Officer': 5,
 'Janitor': 3}

In [220]:
roundstart_jobs = {x: y.get("roundstart", 0) for x,y in rsm._get_stat('manifest').items() if x in prefcounts2}

In [221]:
roundstart_jobs

{'Security Officer': 6,
 'Janitor': 1,
 'Medical Doctor': 3,
 'Scientist': 6,
 'Station Engineer': 2}